In [16]:
import tensorflow as tf
from IPython.core import display
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from math import pi
import kdl as tk
from quaternion import *
from scipy.spatial.transform import Rotation as R

plt.style.use("default")
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
gpu_name=tf.test.gpu_device_name()
cpu_name = '/device:CPU:0'
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

In [17]:
def rotvec2mat(x):
    return R.as_matrix(R.from_rotvec(x))

In [18]:
targets = np.array([[ 0.55, -0.6 ,  0.55],
       [ 0.55, -0.6 ,  0.6 ],
       [ 0.55, -0.6 ,  0.65],
       [ 0.55, -0.6 ,  0.7 ],
       [ 0.55, -0.6 ,  0.75],
       [ 0.55, -0.6 ,  0.8 ],
       [ 0.55, -0.55,  0.55],
       [ 0.55, -0.55,  0.6 ],
       [ 0.55, -0.55,  0.65],
       [ 0.55, -0.55,  0.7 ],
       [ 0.55, -0.55,  0.75],
       [ 0.55, -0.55,  0.8 ],
       [ 0.55, -0.5 ,  0.55],
       [ 0.55, -0.5 ,  0.6 ],
       [ 0.55, -0.5 ,  0.65],
       [ 0.55, -0.5 ,  0.7 ],
       [ 0.55, -0.5 ,  0.75],
       [ 0.55, -0.5 ,  0.8 ],
       [ 0.55, -0.45,  0.55],
       [ 0.55, -0.45,  0.6 ],
       [ 0.55, -0.45,  0.65],
       [ 0.55, -0.45,  0.7 ],
       [ 0.55, -0.45,  0.75],
       [ 0.55, -0.45,  0.8 ],
       [ 0.55, -0.4 ,  0.55],
       [ 0.55, -0.4 ,  0.6 ],
       [ 0.55, -0.4 ,  0.65],
       [ 0.55, -0.4 ,  0.7 ],
       [ 0.55, -0.4 ,  0.75],
       [ 0.55, -0.4 ,  0.8 ],
       [ 0.55, -0.35,  0.55],
       [ 0.55, -0.35,  0.6 ],
       [ 0.55, -0.35,  0.65],
       [ 0.55, -0.35,  0.7 ],
       [ 0.55, -0.35,  0.75],
       [ 0.55, -0.35,  0.8 ],
       [ 0.55, -0.3 ,  0.55],
       [ 0.55, -0.3 ,  0.6 ],
       [ 0.55, -0.3 ,  0.65],
       [ 0.55, -0.3 ,  0.7 ],
       [ 0.55, -0.3 ,  0.75],
       [ 0.55, -0.3 ,  0.8 ],
       [ 0.55, -0.25,  0.55],
       [ 0.55, -0.25,  0.6 ],
       [ 0.55, -0.25,  0.65],
       [ 0.55, -0.25,  0.7 ],
       [ 0.55, -0.25,  0.75],
       [ 0.55, -0.25,  0.8 ],
       [ 0.55, -0.2 ,  0.55],
       [ 0.55, -0.2 ,  0.6 ],
       [ 0.55, -0.2 ,  0.65],
       [ 0.55, -0.2 ,  0.7 ],
       [ 0.55, -0.2 ,  0.75],
       [ 0.55, -0.2 ,  0.8 ],
       [ 0.55,  0.15,  0.55],
       [ 0.55,  0.15,  0.6 ],
       [ 0.55,  0.15,  0.65],
       [ 0.55,  0.15,  0.7 ],
       [ 0.55,  0.15,  0.75],
       [ 0.55,  0.15,  0.8 ],
       [ 0.55,  0.2 ,  0.55],
       [ 0.55,  0.2 ,  0.6 ],
       [ 0.55,  0.2 ,  0.65],
       [ 0.55,  0.2 ,  0.7 ],
       [ 0.55,  0.2 ,  0.75],
       [ 0.55,  0.2 ,  0.8 ],
       [ 0.55,  0.25,  0.55],
       [ 0.55,  0.25,  0.6 ],
       [ 0.55,  0.25,  0.65],
       [ 0.55,  0.25,  0.7 ],
       [ 0.55,  0.25,  0.75],
       [ 0.55,  0.25,  0.8 ],
       [ 0.55,  0.3 ,  0.55],
       [ 0.55,  0.3 ,  0.6 ],
       [ 0.55,  0.3 ,  0.65],
       [ 0.55,  0.3 ,  0.7 ],
       [ 0.55,  0.3 ,  0.75],
       [ 0.55,  0.3 ,  0.8 ],
       [ 0.55,  0.35,  0.55],
       [ 0.55,  0.35,  0.6 ],
       [ 0.55,  0.35,  0.65],
       [ 0.55,  0.35,  0.7 ],
       [ 0.55,  0.35,  0.75],
       [ 0.55,  0.35,  0.8 ],
       [ 0.55,  0.4 ,  0.55],
       [ 0.55,  0.4 ,  0.6 ],
       [ 0.55,  0.4 ,  0.65],
       [ 0.55,  0.4 ,  0.7 ],
       [ 0.55,  0.4 ,  0.75],
       [ 0.55,  0.4 ,  0.8 ]])

In [19]:
targets = targets[6:]

In [20]:
urdf = tk.urdf_from_file('panda_arm_kdl.urdf')
chain = tk.tk_chain_from_urdf(urdf)
dyn = tk.ChainDynParam(chain)


Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_hand']/collision[1]


In [21]:
# For Panda on the floor and others
base_pos = tf.cast(np.array([0., 0., 0.])[None], tf.float32)
base_rot_vec = np.array([0., 0., 0.])
base_rot_mat = tf.cast(rotvec2mat(base_rot_vec), tf.float32)
base_rot_mat_inv = tf.cast(tf.transpose(base_rot_mat), tf.float32)



def kdl2bullet_x(x):
    """
    param: x: [batch, 3] position in kdl (wrt robot base frame)
    """
    return tf.einsum('ij, aj->ai',base_rot_mat, x) + base_pos

def bullet2kdl_x(x):
    """
    param: x: [batch, 3] position in bullet (wrt bullet world frame)
    """
    return tf.einsum('ij, aj->ai',base_rot_mat_inv, x - base_pos)

def kdl2bullet_q(x):
    """
    param: x: [batch, 9] vectorized rotation matrix in kdl (wrt robot base frame)
    """
    kdl_end_rot_mat = tf.reshape(x, (x.shape[0], 3,3))
    world_end_rot_mat =  tf.einsum('ij, ajk->aik', base_rot_mat, kdl_end_rot_mat)
    return rot2quat(world_end_rot_mat)

def bullet2kdl_q(x):
    """
    param: x: [batch, 9] vectorized rotation matrix in bullet (wrt bullet world frame)
    """
    world_end_rot_mat = tf.reshape(x, (x.shape[0], 3,3))
    kdl_end_rot_mat =  tf.einsum('ij, ajk->aik', base_rot_mat_inv, world_end_rot_mat)
    return rot2quat(kdl_end_rot_mat)
        
def kdl2bullet_xq(x):
    """
    param: x: [batch, 12] position + vectorized rotation matrix
    """
    return kdl2bullet_x(x[:, :3]), kdl2bullet_q(x[:, 3:])

def bullet2kdl_xq(x):
    """
    param: x: [batch, 12] position + vectorized rotation matrix
    """
    return bullet2kdl_x(x[:, :3]), bullet2kdl_q(x[:, 3:])


### End-pose cost

In [22]:
quats_xyz = Quaternion(create_unit_rotations(angle=pi/2))
u = tf.constant([0.        , np.pi/2, 0. ])       
theta = tf.linalg.norm(u)
u = u/theta
mu = tf.cast(utils.axis_angle2quats(u, theta), tf.float32)[None]

mu_  = Quaternion(tf.tile(mu, (3,1)))
rot_mu =  mu_ * quats_xyz
u = Quaternion(rot_mu) - mu_
u = u/tf.linalg.norm(u, axis=-1)[:,None]
norms = tf.constant([1, 1e-5, 1e-5])
cov = tf.reduce_sum(tf.einsum('k,kij->kij', 
                                  norms,
                                  tf.einsum('ki, kj->kij',u,u)), axis=0)
cov += 1e-6 * tf.eye(4)

prec = tf.linalg.inv(cov)
mu_quat = mu[None]
def normalS3_log_prob(x):
    """
    x : [hole_nb, batch_nb, 4]
    Quaternion lognormal distribution
    """
    mu__ = Quaternion(tf.tile(mu_quat, (x.q.shape[0], x.q.shape[1], 1)))
    diff = x - mu__
    quad = -0.5*tf.linalg.einsum('hbi,hbi->hb',diff, tf.linalg.einsum('ij,hbj->hbi',prec,diff))
    return quad


cov_x = tf.linalg.diag([1e-3, 1e-3, 1e-3])
prec_x = tf.linalg.inv(cov_x)

def normal_log_prob_(mu_x):
    """
    mu_x : [hole_nb, 1, 3]
    """
    def normal_log_prob(x):
        """
        x: [hole_nb, batch_nb, 3]
        Cartesian lognormal distribution
        """
        diff = x - mu_x
        quad = -0.5*tf.linalg.einsum('hbi,hbi->hb',diff, tf.linalg.einsum('ij,hbj->hbi',prec_x,diff))
        return quad
    return normal_log_prob

def normal_log_prob__(mu_x):
    """
    mu_x : [1, 3]
    """
    def normal_log_prob(x):
        """
        x: [batch_nb, 3]
        Cartesian lognormal distribution
        """
        diff = x - mu_x
        quad = -0.5*tf.linalg.einsum('bi,bi->b',diff, tf.linalg.einsum('ij,bj->bi',prec_x,diff))
        return quad
    return normal_log_prob

### Joint limit costs

In [23]:
joint_limits = tf.stack(chain.joint_limits)
lower_limits = joint_limits[:,0][None, None]
upper_limits = joint_limits[:,1][None, None]
def plot_limits(q):
    plt.plot(joint_limits, "o")
    plt.plot(q, "kx")
    
mean_limits = tf.reduce_mean(joint_limits,-1)[None]
@tf.function
def cost_joint_limit(q):
    rl = tf.math.minimum(q-lower_limits, 0)
    ru = tf.math.maximum(q-upper_limits, 0)
    return 0.5*(tf.linalg.einsum('hbi,hbi->hb',rl, rl) + tf.linalg.einsum('hbi,hbi->hb',ru, ru))


### Optimization 

In [24]:
batch = 1500
mu_x = tf.cast(targets, tf.float32)[:, None]
normal_log_prob = normal_log_prob_(mu_x)
hole_nb = targets.shape[0]
@tf.function
def cost_ik(q):
    """
    q: [hole_nb, batch_nb, nb_dof]
    """
    batch = q.shape[1]
    q_vec = tf.reshape(q, (hole_nb*batch, q.shape[2]))
    xs_ = chain.xs(q_vec, layout=tk.FkLayout.xm)[:, -1]
#     xs_ = tf.reshape(xs_, (q.shape[0], q.shape[1], xs_.shape[-1]))
    
    # if robot base frame is changed:
#     world_end_pos, world_end_quats = kdl2bullet_xq(xs_)
#     else:kdl_end_rot_mat = tf.reshape(x, (x.shape[0], 3,3))
    world_end_pos = tf.reshape(xs_[..., :3], (hole_nb, batch, 3))
    

    world_end_quats = rot2quat(tf.reshape(xs_[...,3:], (hole_nb*batch, 3, 3)))
    world_end_quats = tf.reshape(world_end_quats, (hole_nb, batch, 4))
        
    ee_pos_cost = -normal_log_prob(world_end_pos)
    ee_quat_cost = -normalS3_log_prob(Quaternion(world_end_quats))
    return ee_pos_cost +  ee_quat_cost*0.1 + cost_joint_limit(q)*1e3


In [25]:
start  = tf.Variable(
    tf.random.uniform(shape=(hole_nb, batch, 7), minval=tf.stack(chain.joint_limits)[:,0],maxval=tf.stack(chain.joint_limits)[:,1]))

In [26]:
@tf.function
def cost():
    return cost_ik(start)
optimizer = tf.keras.optimizers.Adam(0.5)


In [27]:
# Stochastic Gradient Descent
for i in range(1000000):
    try :
        optimizer.minimize(cost,[start] )
        display.clear_output(wait=True)
        tf.print(cost())
    except KeyboardInterrupt:
        break

[[1417.85071 2050.7666 1814.39014 ... 674.051758 309.000824 2690.51831]
 [518.345825 220.571808 2822.66113 ... 3771.51855 1234.07678 1295.19141]
 [944.807251 540.703552 14976.5039 ... 3548.83447 2114.05103 973.256592]
 ...
 [1969.25757 1405.12646 2377.56567 ... 252.267166 1374.51074 1762.47363]
 [1713.96606 640.88678 2065.30493 ... 1062.61316 427.608337 63.4384918]
 [988.594 1890.82446 717.499 ... 3084.02954 1236.43982 2355.18604]]


In [13]:
good_q__ = start

In [14]:
good_q_ = [None for i in range(hole_nb)]
max_nb = 1e5
for i in range(hole_nb):
    good_q_[i] = good_q__[i][cost_joint_limit(good_q__)[i] < 1e-2]
    max_nb = min(max_nb,good_q_[i].shape[0])
    
for i in range(hole_nb):
    good_q_[i] = good_q_[i][:max_nb]
    
good_q_ = tf.stack(good_q_)

In [15]:
good_q_.shape

TensorShape([84, 475, 7])

In [ ]:
i = 0 # rivet hole
j = 0 # configuration nb

In [ ]:
plot_limits(good_q_[i][j])
tf.print(cost_joint_limit(good_q_[i][j][None]))
j+=1
# i+=1

In [ ]:
def check_q(q):
    batch = q.shape[1]
    q_vec = tf.reshape(q, (hole_nb*batch, q.shape[2]))
    xs_ = chain.xs(q_vec, layout=tk.FkLayout.xm)[:, -1]
    
    world_end_pos = tf.reshape(xs_[..., :3], (hole_nb, batch , 3))
    world_end_quats = rot2quat(tf.reshape(xs_[...,3:], (hole_nb*batch, 3, 3)))
    world_end_quats = tf.reshape(world_end_quats, (hole_nb, batch, 4))
    
    
    
    good_xs = [None for i in range(hole_nb)]
    good_xs_ind = [None for i in range(hole_nb)]
    max_nb = 1e5
    for i in range(hole_nb):
#         print(world_end_pos[i][normal_log_prob(world_end_pos)[i] > -1e-2].shape)
        good_xs[i] = world_end_pos[i][normal_log_prob(world_end_pos)[i] > -1e-1]
        good_xs_ind[i] = np.where(normal_log_prob(world_end_pos)[i] > -1e-1)
        max_nb = min(max_nb,good_xs[i].shape[0])

    for i in range(hole_nb):
        good_xs[i] = good_xs[i][:max_nb]

    good_xs = tf.stack(good_xs)
    return world_end_pos, good_xs, good_xs_ind
    

In [ ]:
xs, good_xs, good_xs_ind = check_q(good_q_)
good_xs.shape

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(15,5))
N = xs.shape[1]
j = 0 # choose a rivet hole
for i in range(3):
    ax[i].plot(range(N), xs[j, :,i], "r.")
    ax[i].plot(good_xs_ind[j][0][:good_xs.shape[1]], good_xs[j,:,i], "gx", ms=10)
    ax[i].plot(range(N), [mu_x[j,:,i]]*N, "k")

In [ ]:
good_q = [None for i in range(hole_nb)]
max_nb = 1e5
for i in range(hole_nb):
    good_q[i] = good_q_[i][-normal_log_prob(xs)[i] < 1e-1]
    max_nb = min(max_nb,good_q[i].shape[0])
    
for i in range(hole_nb):
    good_q[i] = good_q[i][:max_nb]
    
good_q = tf.stack(good_q)
good_q.shape

In [ ]:
i = 0
j = 1

In [ ]:
plot_limits(good_q[j,i])
tf.print(cost_joint_limit(good_q[j,i][None]))
i+=1

In [ ]:
# check if there is any nan
tf.where(tf.math.is_nan(good_q) == True)

### Displacement cost , new indices paper 2020

In [ ]:
# Force: 
F = 150 # in Newton
nF = tf.constant([-1, 0., 0]) #direction of force in world frame coordinates

In [ ]:
# For 7dof robot
K = tf.eye(chain.nb_joint)*1e5
C = tf.linalg.diag(tf.ones(7)*0.06)

# # ## For 3dof robot in the paper
# K = tf.linalg.diag([2.4*1e5, 1e5, 1e5])
# C = tf.linalg.diag([0.06, 0.06, 0.06])

f = tf.constant(66.) #Hz
w = tf.constant(2*pi*f)
w = tf.constant(150.)

def compute_H(w, M, J, C, K):
    """
    Finds the displacement force frequency response function H(w)
    """
    real = -(w**2)*M+K
    imag = w*C
    comp = tf.complex(real, imag)
#     inv = tf.linalg.inv(comp, adjoint=True)
    try:
        inv = tf.linalg.inv(comp, adjoint=True)
    except:
        r0  = tf.linalg.pinv(real) @ imag
        y11 = tf.linalg.pinv(imag @ r0 + real)
        y10 = -r0 @ y11

        inv = tf.cast(tf.complex(y11,y10), dtype =comp.dtype)
    return J@inv@tf.transpose(J, (0,2,1))

def displacement(q):
        q_vec = tf.reshape(q, (q.shape[0]*q.shape[1], q.shape[2]))
        M = dyn.inertia_matrix(q_vec)
        J = chain.jacobian(q_vec, layout=tk.FkLayout.x)
        J = tf.complex(J, tf.zeros_like(J))
        H = compute_H(w, M, J, C, K)
        
        A = tf.transpose(H, perm=(0,2,1),conjugate=True)@H
        evals, evecs = tf.linalg.eigh(A)

        evals = tf.reshape(evals, (q.shape[0], q.shape[1], 3))
        return tf.math.sqrt(tf.math.abs(tf.math.real(evals)))

@tf.function
def displacement_(w, q):
        q_vec = tf.reshape(q, (q.shape[0]*q.shape[1], q.shape[2]))
        M = dyn.inertia_matrix(q_vec)
        J = chain.jacobian(q_vec, layout=tk.FkLayout.x)
        J = tf.complex(J, tf.zeros_like(J))
        H = compute_H(w, M, J, C, K)
        A = tf.transpose(H, (0,2,1),conjugate=True)@H
        evals, evecs = tf.linalg.eigh(A)
        evals = tf.reshape(evals, (q.shape[0], q.shape[1], 3))
        return  tf.math.sqrt(tf.math.abs(tf.math.real(evals)))

In [ ]:
good_q.shape

In [ ]:
data_nb = 50
config_nb = 500
w__ = 66*2*pi
w__ = 450
w_ = tf.cast(np.linspace(0, w__, data_nb), tf.float32)
disp = np.zeros((hole_nb, config_nb, data_nb))
disp_x = np.zeros((hole_nb, config_nb, data_nb, 3))
disp_x
q_random = tf.random.uniform(shape=(hole_nb, config_nb,7), minval=tf.stack(chain.joint_limits)[:,0],maxval=tf.stack(chain.joint_limits)[:,1])
for i in range(data_nb):
#     d_ = displacement_(w_[i], good_q[:,:config_nb])
    d_ = displacement_(w_[i], q_random[:,:config_nb])
#     disp[:,:, i] = tf.reduce_max(displacement_(w_[i], q_random), -1)
#     disp[:,:, i] = tf.reduce_max(d_, -1)
    disp_x[:,:,i] = d_


In [ ]:
# disp = tf.einsum('abci,i->abc',disp_x, tf.constant([1., 0., 0.]))
disp = tf.reduce_max(disp_x, -1)

In [ ]:
fig,ax = plt.subplots(1, figsize=(4,3))
for j in range(1):
    for i in range(config_nb):
        ax.plot(w_,disp[j,i]*1e3, "-", label="Point A")
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set_xlabel('$\omega$'+' (rad/s)')
ax.set_ylabel('$\eta$'+' (mm/N)')
ax.grid("on")
# Major ticks every 20, minor ticks every 5
major_ticks = np.arange(0, w_[-1], 50)
minor_ticks = np.arange(0, w_[-1], 10)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)

def forward(x):
    return x * 130.


def inverse(x):
    return x/130.

secax = ax.secondary_yaxis('right', functions=(forward, inverse))
secax.set_ylabel('$\Delta x$'+' (mm)')
secax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))


# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
# ax.set_facecolor("w")
# plt.legend()

### Optimization for displacement

In [ ]:
batch = 500
q_test = tf.Variable(good_q[:, :batch], trainable=True)
@tf.function
def cost_(): 
    return cost_ik(q_test)

In [ ]:
@tf.function
def train_step():
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(q_test)
        disp_cost = tf.norm(displacement(q_test),ord=2,axis=-1 )
        loss = cost_ik(q_test) + 1e1*disp_cost
        
    grads = tape.gradient(loss, [q_test])

#     J = chain.jacobian(q_test, layout=tk.FkLayout.xq)
#     Jv = J[:, :3]
#     Jv_pinv = tf.linalg.pinv(Jv)
    
#     Jw = J[:,3:]
#     Jw_ = tf.concat([tf.zeros((batch, 1, chain.nb_joint)), Jw],1)
#     Jq = 0.5*utils.omega_quat_left(kdl_q)@Jw_
#     Jq_pinv = tf.linalg.pinv(Jq)
    
#     dquat = world_q - mu_batch
#     dx = world_end_pos - mu_x
#     grads_ = grads[0] + 10*tf.linalg.matvec(Jq_pinv, dquat) + 100*tf.linalg.matvec(Jv_pinv, dx)
#     grads_ = tf.clip_by_value(grads[0], -1e15, 1e15)
    grads_ = grads[0]
    grads_ = tf.where(tf.math.is_nan(grads_), tf.zeros_like(grads_), grads_)
    optimizer.apply_gradients(zip([grads_], [q_test]))
    return loss, grads

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.05)

In [ ]:
for i in range(1000000):
    try :
        loss, grads = train_step()
        display.clear_output(wait=True)
        tf.print(loss)
    except KeyboardInterrupt:
        break

In [ ]:
# check if there is any nan
tf.where(tf.math.is_nan(loss) == True)

In [ ]:
xs_,_,_ = check_q(q_test)
print(xs_.shape)
i=0

In [ ]:
j = 0

In [ ]:
plt.plot(tf.linalg.norm(displacement(good_q[j,:batch][None])[0],axis=-1), "bx", label="Before optimization")
plt.plot(tf.linalg.norm(displacement(q_test[j][None])[0],axis=-1), "rx", label="After optimization")
plt.legend()
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
j+=1
# do with 3 different w values

In [ ]:
(displacement(good_q[:,:batch])*1e3).numpy()[...,0].flatten().shape

In [ ]:
fig,ax=plt.subplots(1,figsize=(4,3))
density = 1
after_opt_data = displacement(q_test)*1e3
before_opt_data = displacement(good_q[:,:batch])*1e3
# ax.hist(tf.linalg.norm(after_opt_data,axis=-1).numpy().flatten(), density=density, color="b", label="After optimization")
# ax.hist(tf.linalg.norm(before_opt_data,axis=-1).numpy().flatten(), density=density, alpha=0.3, color="r", label="Before optimization")
# ax.plot(after_opt_data.numpy().flatten(), np.ones_like(after_opt_data.numpy().flatten())*(-0.1), "bx", alpha=0.1)
# ax.plot(before_opt_data.numpy().flatten(), np.ones_like(after_opt_data.numpy().flatten())*(-0.2), "rx", alpha=0.1)

ax.hist(after_opt_data.numpy()[...,0].flatten(),density=density,color="b", label="After optimization")
ax.hist(before_opt_data.numpy()[...,0].flatten(), color="r",density=density, alpha=0.3, label="Before optimization")
ax.plot(after_opt_data.numpy()[...,0].flatten(), np.ones_like(after_opt_data.numpy()[...,0].flatten())*(-100), "bx", alpha=0.1)
ax.plot(before_opt_data.numpy()[...,0].flatten(), np.ones_like(after_opt_data.numpy()[...,0].flatten())*(-300), "rx", alpha=0.1)



ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_xlabel('$\eta$'+' (mm/N)')
ax.set_ylabel("density")
secax = ax.secondary_xaxis('top', functions=(forward, inverse))
secax.set_xlabel('$\Delta x$'+' (mm)')
secax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.legend(loc=7)

ax.set_yticks([])

In [ ]:
fig,ax = plt.subplots(ncols=3)
j = 0
for i in range(3):
    ax[i].plot(before_opt_data.numpy()[j,:,i])

In [ ]:
fig,ax = plt.subplots(figsize=(4,3))
density = 1
after_opt_data = displacement(q_test)*1e3
before_opt_data = displacement(good_q[:,:batch])*1e3

ax.boxplot([ before_opt_data.numpy()[...,0].flatten(), after_opt_data.numpy()[...,0].flatten()])

# ax.boxplot([tf.linalg.norm(before_opt_data,axis=-1).numpy().flatten(), 
#             tf.linalg.norm(after_opt_data,axis=-1).numpy().flatten()])

# ax.boxplot(tf.linalg.norm(after_opt_data,axis=-1).numpy().flatten())
ticks = ["Before", "After"]
ax.set_xticks([1,2])
ax.set_xticklabels( ticks, rotation=0 )
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.set_ylabel('$\eta$'+' (mm/N)')

secax = ax.secondary_yaxis('right', functions=(forward, inverse))
secax.set_ylabel('$\Delta x$'+' (mm)')
secax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

In [ ]:
print(np.std(after_opt_data.numpy()[...,0].flatten()))
np.mean(after_opt_data.numpy()[...,0].flatten())

In [ ]:
print(np.std(before_opt_data.numpy()[...,0].flatten()))
np.mean(before_opt_data.numpy()[...,0].flatten())